In [1]:
#First we need to import all the required packages
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
#Now we load the json file and extract the required data.
with open('Desktop\intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [3]:
#we use “LabelEncoder()” function provided by scikit-learn to convert the target labels into a model understandable form.
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
#model training
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [6]:
#training our model
epochs = 700
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/700
2/2 [==============================] - 1s 8ms/step - loss: 2.0788 - accuracy: 0.1212
Epoch 2/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0773 - accuracy: 0.1212
Epoch 3/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0763 - accuracy: 0.1212
Epoch 4/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0755 - accuracy: 0.1212
Epoch 5/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0749 - accuracy: 0.1212
Epoch 6/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0740 - accuracy: 0.1212
Epoch 7/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0731 - accuracy: 0.1212
Epoch 8/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0725 - accuracy: 0.1212
Epoch 9/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0719 - accuracy: 0.1212
Epoch 10/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0710 - accuracy: 0.1212
Epoch 11/700
2/2 [===========

In [7]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("Desktop\intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
ChatBot: Hi
User: who are you?
ChatBot: I.m Joana, your bot assistant
User: nice to meet you
ChatBot: Tell me how can assist you
User: matte
ChatBot: My pleasure
User: how to open an acount
ChatBot: Tell me your problem to assist you
User: how to open an account
ChatBot: You can just easily create a new account from our web site
User: account
ChatBot: Tell me your problem to assist you
User: account
ChatBot: Yes Sure, How can I support you
User: open account
ChatBot: Tell me your problem to assist you
User: how to start account
ChatBot: Tell me how can assist you
User: how to open an account
ChatBot: Just go to our web site and follow the guidelines to create a new account
User: hello
ChatBot: Hi there
User: Thanks for the help
ChatBot: My pleasure
User: What are you?
ChatBot: I'm Joana, an Artificial Intelligent bot
User: Could you help me?
ChatBot: Tell me how can assist you
User: give me a hand please
ChatBot: Tell me how ca